In [1]:
import pandas as pd
import numpy as np
import os
from math import pi, atan, sinh, cos, log, tan, radians, degrees
import numpy as np
from PIL import Image,ImageDraw
from datetime import datetime,timedelta
import math


In [2]:
zoom = 8.47
image_size = (719,719)

In [3]:

## extract the frames data
def latlon_to_pixel(lat, lon, zoom):
    """Convert latitude and longitude to pixel coordinates at the specified zoom level."""
    lat_rad = radians(lat)
    n = 2.0 ** zoom
    x = (lon + 180.0) / 360.0 * n * 256.0
    y = (1.0 - log(tan(lat_rad) + (1 / cos(lat_rad))) / pi) / 2.0 * n * 256.0
    return x, y

def pixel_to_latlon(x, y, zoom):
    """Convert pixel coordinates to latitude and longitude at the specified zoom level."""
    n = 2.0 ** zoom
    lon = x / (n * 256.0) * 360.0 - 180.0
    lat_rad = atan(sinh(pi * (1 - 2 * y / (n * 256.0))))
    lat = degrees(lat_rad)
    return lat, lon

def generate_latlon_grid(center_lat, center_lon, zoom, image_size):
    """Generate a grid of latitude and longitude values for each pixel in the image."""
    width, height = image_size

    # Get pixel coordinates of the center
    center_x, center_y = latlon_to_pixel(center_lat, center_lon, zoom)

    # Calculate pixel offsets
    half_width = width // 2
    half_height = height // 2

    latlon_grid = np.zeros((height, width, 2))

    for y in range(height):
        for x in range(width):
            pixel_x = center_x - half_width + x
            pixel_y = center_y - half_height + y
            lat, lon = pixel_to_latlon(pixel_x, pixel_y, zoom)
            latlon_grid[y, x] = (lat, lon)
    
    return latlon_grid

def rgb_to_hex(rgb):
    """Convert an RGB tuple to a hex string."""
    return '#{:02x}{:02x}{:02x}'.format(rgb[0], rgb[1], rgb[2])



In [51]:
image = Image.open('/home/tensor/vaib/imd_radar/target_processed/delhi/ppi/ppi_delhi_20240813T1340.png')
image = image.resize(image_size)  # Ensure the image is 512x512 pixels
pixels = np.array(image)
latlon_grid = generate_latlon_grid(28.7041, 77.1025, zoom, image_size)
data = []
# Print latitude, longitude, and hex color value of every pixel
for y in range(image_size[1]):
    for x in range(image_size[0]):
        lat, lon = latlon_grid[y, x]
        color = pixels[y, x]
        # hex_color = rgb_to_hex(color)
        data.append([x, y, lat, lon, color])

df = pd.DataFrame(data, columns=['Pixel_X', 'Pixel_Y', 'latitude', 'longitude', 'Color'])

In [52]:
df['lat_diff'] = (df['latitude'] - 28.5433498).abs()
df['lon_diff'] = (df['longitude'] - 77.187352).abs()
df['omega'] = df['lat_diff'] + df['lon_diff']
# df['localityId'] = row['localityId']
df.sort_values('omega')[:1]

,Pixel_X,Pixel_Y,latitude,longitude,Color,lat_diff,lon_diff,omega
291575,380,405,28.543966,77.185783,"[0, 0, 0, 0]",0.000616,0.001569,0.002185


In [64]:
img = Image.open('/home/ubuntu/vaib/imd_radar/target_processed/delhi/ppi/ppi_delhi_20240808T2010.png')


img = img.convert("RGBA")
draw = ImageDraw.Draw(img)
x, y = 393,	390 # Replace with your coordinates
radius = 0.5
pixels = img.load()
draw.ellipse((x-radius, y-radius, x+radius, y+radius), fill=(0, 0, 0))

img.save('/home/ubuntu/vaib/imd_radar/target_processed/delhi/ppi/ppi_delhi_20240808T2010eeeeee.png')

IIT Delhi | 0.003128
CP Central Garden 28.6440, 77.1885 | 0.004157
Noida GIP mall 28.5675, 77.3262 | 0.003126
Pacific Mall Punjabi Baug 28.6417, 77.1067 | 0.001547
 

In [36]:
np.average([0.003128,0.004157,0.003126,0.001547])

np.float64(0.0029895)

In [6]:
import math

def lat_lon_to_pixel(latitude, longitude, zoom, img_width, img_height, center_lat, center_lon):
    # Constants
    origin_shift = 2 * math.pi * 6378137 / 2.0
    initial_resolution = 2 * math.pi * 6378137 / 256.0
    
    # Convert latitude/longitude to Mercator coordinates
    def lat_lon_to_meters(lat, lon):
        mx = lon * origin_shift / 180.0
        my = math.log(math.tan((90 + lat) * math.pi / 360.0)) / (math.pi / 180.0)
        my = my * origin_shift / 180.0
        return mx, my

    # Get the map center in Mercator coordinates
    center_mx, center_my = lat_lon_to_meters(center_lat, center_lon)
    
    # Convert the given latitude/longitude to Mercator coordinates
    mx, my = lat_lon_to_meters(latitude, longitude)
    
    # Calculate the resolution (meters per pixel) for the given zoom level
    resolution = initial_resolution / (2 ** zoom)
    
    # Calculate pixel offsets from the center
    pixel_x_offset = (mx - center_mx) / resolution
    pixel_y_offset = (center_my - my) / resolution
    
    # Calculate pixel coordinates relative to the top-left corner of the image
    pixel_x = img_width / 2 + pixel_x_offset
    pixel_y = img_height / 2 + pixel_y_offset
    
    # Round to nearest pixel
    pixel_x = int(round(pixel_x))
    pixel_y = int(round(pixel_y))
    
    return pixel_x, pixel_y

# Example usage:
latitude = 28.6440  # Replace with the latitude
longitude =  77.1885 # Replace with the longitude
zoom_level = 8.44
img_width = 566
img_height = 566
center_lat = 28.58967  # Center latitude
center_lon = 77.22196  # Center longitude

pixel_x, pixel_y = lat_lon_to_pixel(latitude, longitude, zoom_level, img_width, img_height, center_lat, center_lon)
print(f"Pixel coordinates: ({pixel_x}, {pixel_y})")

Pixel coordinates: (275, 268)


In [9]:
import math

def lat_lon_to_pixel(latitude, longitude, zoom, img_width, img_height, center_lat, center_lon):
    # Constants
    origin_shift = 2 * math.pi * 6378137 / 2.0
    initial_resolution = 2 * math.pi * 6378137 / 256.0
    
    # Convert latitude/longitude to Mercator coordinates
    def lat_lon_to_meters(lat, lon):
        mx = lon * origin_shift / 180.0
        my = math.log(math.tan((90 + lat) * math.pi / 360.0)) / (math.pi / 180.0)
        my = my * origin_shift / 180.0
        return mx, my

    # Get the map center in Mercator coordinates
    center_mx, center_my = lat_lon_to_meters(center_lat, center_lon)
    
    # Convert the given latitude/longitude to Mercator coordinates
    mx, my = lat_lon_to_meters(latitude, longitude)
    
    # Calculate the resolution (meters per pixel) for the given zoom level
    resolution = initial_resolution / (2 ** zoom)
    
    # Calculate pixel offsets from the center
    pixel_x_offset = (mx - center_mx) / resolution
    pixel_y_offset = (center_my - my) / resolution
    
    # Calculate pixel coordinates relative to the top-left corner of the image
    pixel_x = img_width / 2 + pixel_x_offset
    pixel_y = img_height / 2 + pixel_y_offset
    
    # Round to nearest pixel
    pixel_x = int(round(pixel_x))
    pixel_y = int(round(pixel_y))
    
    return pixel_x, pixel_y

# Example usage:
latitude = 28.6440  # Replace with the latitude
longitude =  77.1885   # Replace with the longitude
zoom_level = 8.47
img_width = 566
img_height = 566
center_lat = 28.58967  # Center latitude
center_lon = 77.22196  # Center longitude

pixel_x, pixel_y = lat_lon_to_pixel(latitude, longitude, zoom_level, img_width, img_height, center_lat, center_lon)
print(f"Pixel coordinates: ({pixel_x}, {pixel_y})")


Pixel coordinates: (275, 267)


In [16]:
import math

# Constants
image_size = 719  # in pixels
map_size_km = 150  # in kilometers
center_lat = 28.58967
center_lon = 77.22196
km_per_pixel = map_size_km / image_size

# Radius of the Earth in kilometers
R = 6371.0

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    distance = R * c
    return distance

def calculate_pixel_coordinates(lat, lon):
    # Calculate the distance from the center to the point
    dist_lat = haversine(center_lat, center_lon, lat, center_lon)
    dist_lon = haversine(center_lat, center_lon, center_lat, lon)
    
    # Adjust the distances according to direction
    if lat < center_lat:
        dist_lat = -dist_lat
    if lon < center_lon:
        dist_lon = -dist_lon
    
    # Convert distances to pixel offsets
    pixel_offset_x = dist_lon / km_per_pixel
    pixel_offset_y = dist_lat / km_per_pixel
    
    # Calculate the pixel coordinates on the image
    center_pixel = image_size / 2
    pixel_x = center_pixel + pixel_offset_x
    pixel_y = center_pixel - pixel_offset_y  # y decreases as latitude increases
    
    return int(pixel_x), int(pixel_y)

# Example usage
lat = 28.543349 # Latitude of the point
lon =  77.187352    # Longitude of the point

pixel_x, pixel_y = calculate_pixel_coordinates(lat, lon)
print(f"The pixel coordinates for the point ({lat}, {lon}) are: ({pixel_x}, {pixel_y})")


The pixel coordinates for the point (28.543349, 77.187352) are: (343, 384)


In [ ]:
# 28.5433498, 77.187352

import math

def lat_lon_to_pixel(latitude, longitude, zoom, img_width, img_height, center_lat, center_lon):
    # Constants
    origin_shift = 2 * math.pi * 6378137 / 2.0
    initial_resolution = 2 * math.pi * 6378137 / 256.0
    
    # Convert latitude/longitude to Mercator coordinates
    def lat_lon_to_meters(lat, lon):
        mx = lon * origin_shift / 180.0
        my = math.log(math.tan((90 + lat) * math.pi / 360.0)) / (math.pi / 180.0)
        my = my * origin_shift / 180.0
        return mx, my

    # Get the map center in Mercator coordinates
    center_mx, center_my = lat_lon_to_meters(center_lat, center_lon)
    
    # Convert the given latitude/longitude to Mercator coordinates
    mx, my = lat_lon_to_meters(latitude, longitude)
    
    # Calculate the resolution (meters per pixel) for the given zoom level
    resolution = initial_resolution / (2 ** zoom)
    
    # Calculate pixel offsets from the center
    pixel_x_offset = (mx - center_mx) / resolution
    pixel_y_offset = (center_my - my) / resolution
    
    # Calculate pixel coordinates relative to the top-left corner of the image
    pixel_x = img_width / 2 + pixel_x_offset
    pixel_y = img_height / 2 + pixel_y_offset
    
    # Round to nearest pixel
    pixel_x = int(round(pixel_x))
    pixel_y = int(round(pixel_y))
    
    return pixel_x, pixel_y

In [79]:
def get_color_from_image(file_name,x,y,image_size):
    image = Image.open(file_name)
    image = image.resize(image_size)  
    pixels = np.array(image)
    color = pixels[x,y]
    return color

In [130]:

center_lat = 28.58967  # Center latitude
center_lon = 77.22196  # Center longitude

In [135]:
## get list of all the processed images
all_processed = []
home_path = '/home/tensor/vaib/imd_radar'
processed_source = os.path.join(home_path,'target_processed')

for location in os.listdir(processed_source):
    if location == 'delhi':
        for variable in os.listdir(os.path.join(processed_source,location)):
            for file in os.listdir(os.path.join(processed_source,location,variable)):
                if file not in all_processed:
                    all_processed.append(file)
df_imgs = pd.DataFrame({'files':all_processed})

In [131]:
def get_timestamp(x):
    return datetime.strptime(x,'%Y%m%dT%H%M')

In [136]:
# df_processed
df_imgs['var'] = df_imgs['files'].str.split("_").str[0]
df_imgs['location'] = df_imgs['files'].str.split("_").str[1]
df_imgs['timestamp'] = df_imgs['files'].str.split("_").str[2].str[:-4]
df_imgs['timestamp'] = df_imgs['timestamp'].apply(get_timestamp)


In [138]:
df_imgs.sort_values('timestamp',ascending=False,inplace=True)

In [143]:
df_imgs['file_path'] = processed_source + "/" + df_imgs['location'] +"/" + df_imgs['var']+"/" + df_imgs['files']


'/home/tensor/vaib/imd_radar/target_processed/delhi/caz/caz_delhi_20240812T2110.png'

In [44]:
# pwd

In [35]:
## radar scale data
home_path = '/home/ubuntu/vaib/imd_radar'
radar_colors = pd.read_csv(f'{home_path}/radar_colors_delhi.csv')
# radar_colors.info()
radar_colors['rgb'] = radar_colors['rgb'].str[1:].str[:-1]
radar_colors['rgb'] = radar_colors.loc[:,['r','g','b']].values.tolist()
# radar_colors.to_csv('radar_colors_delhi.csv',index=False)


In [47]:
## load sites
sites = pd.read_csv(f'{home_path}/delhi_sites.csv')
sites = sites.loc[~sites['latitude'].isna(),:]
# sites

In [ ]:
image_widths = {'caz':566,'ppi':719,'sri':719}
zoom_levels = {'caz':8.47,'ppi':9.54,'sri':9.54}
map_size = {'caz':250,'ppi':150,'sri':150}
class ExtractColor:
    def __init__(self,var,timestamp,file_name) -> None:
        self.var = var
        self.timestamp = timestamp
        self.file_name = file_name
        self.image_size = image_widths[var]
        self.map_size = map_size[var]
        
        self.R = 6371.0
        self.center_lat = 28.58967
        self.center_lon = 77.22196
        
    def haversine(self,lat1, lon1, lat2, lon2):
        # Convert latitude and longitude from degrees to radians
        lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
        
        # Haversine formula
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        
        distance = R * c
        return distance
        
    def calculate_pixel_coordinates(self,lat, lon):
        # Calculate the distance from the center to the point
        dist_lat = haversine(self.center_lat, self.center_lon, lat, self.center_lon)
        dist_lon = haversine(self.center_lat, self.center_lon, self.center_lat, lon)
        
        # Adjust the distances according to direction
        if lat < self.center_lat:
            dist_lat = -dist_lat
        if lon < self.center_lon:
            dist_lon = -dist_lon
        
        # Convert distances to pixel offsets
        pixel_offset_x = dist_lon / km_per_pixel
        pixel_offset_y = dist_lat / km_per_pixel
        
        # Calculate the pixel coordinates on the image
        center_pixel = image_size / 2
        pixel_x = center_pixel + pixel_offset_x
        pixel_y = center_pixel - pixel_offset_y  # y decreases as latitude increases
        
        return int(pixel_x), int(pixel_y)


In [60]:
# get the pixels for sites lat lon

image_size = 719  # in pixels
map_size_km = 150  # in kilometers
center_lat = 28.58967
center_lon = 77.22196
km_per_pixel = map_size_km / image_size

# Radius of the Earth in kilometers
R = 6371.0

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    distance = R * c
    return distance

def calculate_pixel_coordinates(lat, lon,center_lat,center_lon,km_per_pixel):
    # Calculate the distance from the center to the point
    dist_lat = haversine(center_lat, center_lon, lat, center_lon)
    dist_lon = haversine(center_lat, center_lon, center_lat, lon)
    
    # Adjust the distances according to direction
    if lat < center_lat:
        dist_lat = -dist_lat
    if lon < center_lon:
        dist_lon = -dist_lon
    
    # Convert distances to pixel offsets
    pixel_offset_x = dist_lon / km_per_pixel
    pixel_offset_y = dist_lat / km_per_pixel
    
    # Calculate the pixel coordinates on the image
    center_pixel = image_size / 2
    pixel_x = center_pixel + pixel_offset_x
    pixel_y = center_pixel - pixel_offset_y  # y decreases as latitude increases
    
    return int(pixel_x), int(pixel_y)

In [58]:
image_caz = '/home/ubuntu/vaib/imd_radar/target_processed/delhi/caz/caz_delhi_20240808T0450.png'
image_ppi = '/home/ubuntu/vaib/imd_radar/target_processed/delhi/ppi/ppi_delhi_20240808T0450.png'
image_sri = '/home/ubuntu/vaib/imd_radar/target_processed/delhi/sri/sri_delhi_20240808T0450.png'
# os.path.exists(image_sri)
image_widths = {'caz':566,'ppi':719,'sri':719}
zoom_levels = {'caz':8.47,'ppi':9.54,'sri':9.54}
map_size = {'caz':250,'ppi':150,'sri':150}

def process_image(var,latitude,longitude):
    R = 6371.0 ## radius of earth
    image_size = image_widths[var]  # in pixels
    map_size_km = map_size[var]  # in kilometers
    center_lat = 28.58967
    center_lon = 77.22196
    km_per_pixel = map_size_km / image_size
    x,y = calculate_pixel_coordinates(lat=latitude,lon=longitude,center_lat=center_lat,
                                      center_lon=center_lon,km_per_pixel=km_per_pixel)

    return x,y
    

In [57]:
vars = ['caz','ppi','sri']
coords = ['x','y']

for x in vars:
    for y in coords:
        sites[f"{x}_{y}"] = None

In [62]:
# all_x_y = {}
import math
for index,row in sites.iterrows():
    for var in vars:
        x,y = process_image(var = var,latitude=row['latitude'],longitude=row['longitude'])
        sites.loc[sites['localityId']==row['localityId'],[f"{var}_x",f"{var}_y"]] = [x,y]
        

In [74]:
all_files = []
def get_timestamp(x):
    return datetime.strptime(x,'%Y%m%dT%H%M')
home_path = os.getcwd()
processed_source = os.path.join(home_path,'target_processed')
for location in os.listdir(processed_source):
    if location == "delhi":
        for var in os.listdir(os.path.join(processed_source,location)):
            for file in os.listdir(os.path.join(processed_source,location,var)):
                if file not in all_files:
                    all_files.append(file)

df_imgs = pd.DataFrame({'files':all_files})
df_imgs['var'] = df_imgs['files'].str.split("_").str[0]
df_imgs['location'] = df_imgs['files'].str.split("_").str[1]
df_imgs['timestamp'] = df_imgs['files'].str.split("_").str[2].str[:-4]
df_imgs['timestamp'] = df_imgs['timestamp'].apply(get_timestamp)
df_imgs.sort_values('timestamp',ascending=False,inplace=True)

# df_imgs

In [142]:
sample = df_imgs.loc[df_imgs['timestamp']=='2024-08-08 20:10:00',:]


# def get_reflectivity(dbz):
#     return 10**(dbz/10)
# def rain_intensity_calc(month,z):
#     if month >= 3 and month <= 10:
#         a = 300
#         b = 1.5
#     elif month in (11,12,1,2):
#         a = 200
#         b = 1.2
#     return (z/a)**(1/b)


In [146]:
# data_df = pd.DataFrame()
for index,row in sample.iterrows():
    timestamp = row['timestamp']
    date_month = timestamp.month
    var = row['var']
    file_name = row['files']
    location = row['location']
    file_path = os.path.join(processed_source,location,var,file_name)
    data_df = None
    # print(os.path.exists(file_path))
    for st_index,st_row in sites.iterrows():
        lat = st_row['latitude']
        lon = st_row['longitude']
        locality_id = st_row['localityId']
        
        image = Image.open(file_path)
        key_x = f"{var}_x"
        key_y = f"{var}_y"
        x = st_row[key_x]
        y = st_row[key_y]

        # Get the pixel color value at (x, y)
        pixel_color = image.getpixel((x, y))
        rgb = [pixel_color[0],pixel_color[1],pixel_color[2]]
        a = pixel_color[3]
        color_data = radar_colors[radar_colors['rgb'].apply(lambda x: x == rgb)]
        if len(color_data)>0 and a==255:
            var_data = color_data[var].iloc[0]
            # print(var_data)
        else:
            # print(a)
            var_data = 0
        # print(f"{var_data},{var}")
        if type(data_df)==type(None):
            # data_df['timestamp'] = timestamp
            data_df = pd.DataFrame({'timestamp':[timestamp],var:[var_data],'localityId':[locality_id]})

        else:
            new_df = pd.DataFrame({'timestamp':[timestamp],var:[var_data],'localityId':[locality_id]})
            data_df = pd.concat([data_df,new_df],ignore_index=True)
    # if var == 'ppi' or var == 'caz':
    #     data_df['Z'] = data_df[var].apply(get_reflectivity)
    #     data_df['rain_intensity'] = data_df.apply(lambda x:rain_intensity_calc(date_month,x.Z),axis=1)
    # data_df['timestamp'] = timestamp
    print(data_df)
        # print(f"{timestamp},{var_data},{locality_id}")

# print(data_df)

              timestamp  sri localityId
0   2024-08-08 20:10:00  0.0  ZWL005764
1   2024-08-08 20:10:00  0.0  ZWL008752
2   2024-08-08 20:10:00  0.0  ZWL005996
3   2024-08-08 20:10:00  0.0  ZWL005243
4   2024-08-08 20:10:00  0.0  ZWL009032
..                  ...  ...        ...
119 2024-08-08 20:10:00  0.0  ZWL008401
120 2024-08-08 20:10:00  0.0  ZWL001758
121 2024-08-08 20:10:00  4.3    RRRRRRR
122 2024-08-08 20:10:00  4.3    VVVVVVV
123 2024-08-08 20:10:00  0.0   FZFZFZFZ

[124 rows x 3 columns]
              timestamp  ppi localityId
0   2024-08-08 20:10:00    6  ZWL005764
1   2024-08-08 20:10:00    6  ZWL008752
2   2024-08-08 20:10:00    6  ZWL005996
3   2024-08-08 20:10:00    6  ZWL005243
4   2024-08-08 20:10:00    0  ZWL009032
..                  ...  ...        ...
119 2024-08-08 20:10:00    0  ZWL008401
120 2024-08-08 20:10:00   10  ZWL001758
121 2024-08-08 20:10:00   30    RRRRRRR
122 2024-08-08 20:10:00   18    VVVVVVV
123 2024-08-08 20:10:00    2   FZFZFZFZ

[124 rows x 3 c

In [140]:
# some_df = pd.DataFrame()
# data_df['localityId'].value_counts()
# for x in range(50):
#     if len(some_df) == 0:
#         some_df = pd.DataFrame({'time':['one'],'x':[x]})
#     else:
#         new_df = pd.DataFrame({'time':['two'],'x':[x]})

#         some_df = pd.concat([some_df,new_df],ignore_index=True)
# print(some_df)

localityId
ZWL005764    1
ZWL001933    1
ZWL001186    1
ZWL002908    1
ZWL006720    1
            ..
ZWL007284    1
ZWL009834    1
ZWL007224    1
ZWL003241    1
FZFZFZFZ     1
Name: count, Length: 124, dtype: int64

In [94]:
radar_colors[radar_colors['rgb'].apply(lambda x: x == [58,0,160])]

,rgb,caz,ppi,sri,r,g,b
0,"[58, 0, 160]",16.5,2,4.3,58,0,160
